In [1]:
import os

import pandas as pd

# Target Trial Emulation in Python

On this notebook we will be following the tutorial [*Target Trial Emulation in R*](https://rpubs.com/alanyang0924/TTE) by Alan Yang and translate the **R** code to **Python**.

But before we do, since the tutorial is using a package [`TrialEmulation`](https://github.com/Causal-LDA/TrialEmulation/blob/v0.0.4.2/R/) that is not available in Python as of the making of this notebook, we will first have to simulate the class `trial_sequence`. We will not simulate everything from the package, only the functions that are used in the tutorial.

In [2]:
from trial_sequence import trial_sequence
from trial_sequence.te_weights import show_weight_models
from trial_sequence.utils import stats_glm_logit, save_to_datatable

We separated the logic of [`trial_sequence`](trial_sequence) for readability.

## 1. Setup

First, we have to identify what estimand will be used. For simplicity we will just follow what the tutorial used, which is **per-protocol (PP)** and **intention-to-treat (ITT)**; besides these two, there is also **as-treated (AT)**.

In [3]:
trial_pp = trial_sequence(estimand="PP")
trial_itt = trial_sequence(estimand="ITT")

Let's also make sure that we have dedicated directories to save the files for later inspection.

In [4]:
trial_pp_dir = os.path.join(os.getcwd(), "trial_pp")
os.makedirs(trial_pp_dir, exist_ok=True)

trial_itt_dir = os.path.join(os.getcwd(), "trial_itt")
os.makedirs(trial_itt_dir, exist_ok=True)

## 2. Data Preparation

In [5]:
data_censored = pd.read_csv("data_censored.csv")
data_censored.groupby("id").first().head()

,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible
id,,,,,,,,,,,
1,0,1,1,1.146148,0,0.734203,36,0.083333,0,0,1
2,0,0,1,-0.802142,0,-0.990794,26,-0.750000,0,0,1
3,0,1,0,0.571029,1,0.391966,48,1.083333,0,0,1
4,0,0,0,-0.107079,1,-1.613258,29,-0.500000,0,0,1
5,0,1,1,0.749092,0,1.620330,32,-0.250000,0,0,1


In [6]:
trial_pp.set_data(
    id="id",
    period="period",
    outcome="outcome",
    eligible="eligible",
    treatment="treatment",
    data=data_censored
)

trial_itt.set_data(
    id="id",
    period="period",
    outcome="outcome",
    eligible="eligible",
    treatment="treatment",
    data=data_censored
)

trial_itt


Trial Sequence Object
Estimand: Intent-to-Treat

Data:
 - N: 725 observations from 89 patients
           x2  age  period  outcome  x1     age_s  eligible        x4  \
0    1.146148   36       0        0   1  0.083333         1  0.734203   
1    0.002200   37       1        0   1  0.166667         0  0.734203   
2   -0.481762   38       2        0   0  0.250000         0  0.734203   
3    0.007872   39       3        0   0  0.333333         0  0.734203   
4    0.216054   40       4        0   1  0.416667         0  0.734203   
..        ...  ...     ...      ...  ..       ...       ...       ...   
719  1.650478   67       2        0   0  2.666667         0  0.575268   
720 -0.747906   68       3        0   0  2.750000         0  0.575268   
721 -0.790056   69       4        0   0  2.833333         0  0.575268   
722  0.387429   70       5        0   1  2.916667         0  0.575268   
723 -0.033762   71       6        0   1  3.000000         0  0.575268   

     censored  x3  treatmen

## 3. Weight Models and Censoring

The tutorial used inverse probability of censoring weights (IPCW) to adjust for the effects of informative censoring. To estimate these weights, it constructed time-to-censoring event models and fit two sets of models: one for censoring due to deviation from the assigned treatment, and another for other forms of informative censoring.

### 3.1 Censoring Due to Treatment Switching

The tutorial demonstrates how to set up model formulas for estimating the probability of receiving treatment in the current period. It fits separate models for patients who received treatment $(treatment = 1)$ and those who did not $(treatment = 0)$ in the previous period. To obtain stabilized weights, the approach involves fitting both numerator and denominator models.

Also, the tutorial outlines optional arguments that allow you to specify columns to include or exclude observations from the treatment models. This can be particularly useful when a patient is unable to deviate from a particular treatment assignment during a given period.

In [7]:
trial_pp.set_switch_weight_model(
    numerator="age",
    denominator="age + x1 + x3",
    model_fitter=stats_glm_logit(save_path=os.path.join(trial_pp_dir, "switch_models"))
)

trial_pp.switch_weights

 - Numerator formula: treatment ~ age
 - Denominator formulat: treatment ~ age + x1 + x3
 - Model fitter type: te_stats_glm_logit
 - Weight models not fitted. Use calculate_weights()

If we attempted this function on a ITT estimand, the function will raise an error.

### 3.2 Other Informative Censoring

The tutorial introduced that if there’s additional informative censoring in the data, you can build similar models to estimate the inverse probability of censoring weights (IPCW). This method works for all estimands, and you simply need to specify the `censor_event` column as the censoring indicator.

In [8]:
trial_pp.set_censor_weight_model(
    censor_event="censored",
    numerator="x2",
    denominator="x2 + x1",
    pool_models=None,
    model_fitter=stats_glm_logit(save_path=os.path.join(trial_pp_dir, "switch_models"))
)

trial_pp.censor_weights

 - Numerator formula: 1 - censored ~ x2
 - Denominator formulat: 1 - censored ~ x2 + x1
 - Model fitter type: te_stats_glm_logit
 - Weight models not fitted. Use calculate_weights()

In [9]:
trial_itt.set_censor_weight_model(
    censor_event="censored",
    numerator="x2",
    denominator="x2 + x1",
    pool_models="numerator",
    model_fitter=stats_glm_logit(save_path=os.path.join(trial_itt_dir, "switch_models"))
)

trial_itt.censor_weights

 - Numerator formula: 1 - censored ~ x2
 - Denominator formulat: 1 - censored ~ x2 + x1
 - Numerator model is pooled across treatment arms. Denominator model is not pooled.
 - Model fitter type: te_stats_glm_logit
 - Weight models not fitted. Use calculate_weights()

## 4. Calculate Weights

The tutorial then demonstrates how to fit each individual model and merge them into a set of weights using the `calculate_weights()` function.

In [10]:
trial_pp.calculate_weights()
trial_itt.calculate_weights()

The full object models and summaries are stored in the directories that we created from [step 1](#1-setup). We can then print the result as such:

In [11]:
show_weight_models(trial_itt)

Weight Models for Informative Censoring
---------------------------------------

[[n]]
Model: P(censor_event = 0 | X) for numerator

       Coef.     Std.Err.            z    P>|z|        [0.025       0.975]
2.656607e+01 13424.973712 1.978854e-03 0.998421 -26285.898900 26339.031037
5.549933e-15 13267.631763 4.183062e-19 1.000000 -26004.080416 26004.080416

                  0                1               2           3
             Model:              GLM            AIC:      4.0000
     Link Function:            Logit            BIC:  -4761.8021
Dependent Variable:        Intercept Log-Likelihood: -2.1018e-09
              Date: 2025-03-10 00:35        LL-Null:      0.0000
  No. Observations:              725       Deviance:  4.2061e-09
          Df Model:                1   Pearson chi2:    2.10e-09
      Df Residuals:              723          Scale:      1.0000
            Method:             IRLS                            

                                                       

In [12]:
show_weight_models(trial_pp)

Weight Models for Informative Censoring
---------------------------------------

[[n0]]
Model: P(censor_event = 0 | X, previous treatment = 0) for numerator

       Coef.     Std.Err.            z   P>|z|        [0.025       0.975]
2.556607e+01 16581.144002 1.541876e-03 0.99877 -32472.878998 32524.011135
1.604272e-14 17442.895871 9.197282e-19 1.00000 -34187.447692 34187.447692

                  0                1               2           3
             Model:              GLM            AIC:      4.0000
     Link Function:            Logit            BIC:   -862.8141
Dependent Variable:        Intercept Log-Likelihood: -1.3402e-09
              Date: 2025-03-10 00:35        LL-Null:      0.0000
  No. Observations:              170       Deviance:  2.6809e-09
          Df Model:                1   Pearson chi2:    1.34e-09
      Df Residuals:              168          Scale:      1.0000
            Method:             IRLS                            

                                 

## 5. Specify Outcome Model

The tutorial then shows how to specify the outcome model, where you can incorporate adjustment terms for any variables in the dataset. It automatically includes the numerator terms from the stabilized weight models in the outcome model's formula.

In [13]:
trial_pp.set_outcome_model()
trial_itt.set_outcome_model()

## 6. Expand Trials

The tutorial now proceeds to generate the dataset containing the full sequence of target trials.

In [14]:
trial_pp.set_expansion_options(
    output = save_to_datatable(),
    chunk_size = 500 # the number of patients to include in each expansion iteration
)
trial_itt.set_expansion_options(
    output=save_to_datatable(),
    chunk_size=500
)

### 6.1 Create Sequence of Trials Data

In [15]:
trial_pp.expand_trials()
trial_itt.expand_trials()

trial_pp.expansion

Sequence of Trials Data:
- Chunk Size: 500
- Censor at switch: True
- First period: 0 | Last period: inf

A TE Datastore Datatable object
N: 320 observations
           x2  weight  age  outcome  assigned_treatment  followup_time  \
0    1.146148     NaN   36      0.0                 1.0              0   
1    1.146148     NaN   36      0.0                 1.0              3   
2    1.146148     NaN   36      0.0                 1.0              2   
3    1.146148     NaN   36      0.0                 1.0              4   
4    1.146148     NaN   36      0.0                 1.0              1   
..        ...     ...  ...      ...                 ...            ...   
315  0.687126     NaN   40      0.0                 1.0              0   
316 -1.085325     NaN   48      0.0                 1.0              0   
317 -1.085325     NaN   48      0.0                 1.0              1   
318  0.621108     NaN   36      0.0                 0.0              0   
319 -0.346378     NaN   65  

## 7. Load or Sample from Expanded Data

With the expanded dataset ready, the tutorial demonstrates how to prepare the data for fitting the outcome model. If the dataset fits comfortably in memory, this can be done easily using load_expanded_data.

For larger datasets, sampling may be necessary. This can be controlled using the p_control argument, which sets the probability of including observations where $outcome = 0$. A seed can be specified for reproducibility, and additional filters can be applied, such as selecting specific periods (e.g., $period = 1:60$) or subsetting based on conditions like $age > 65$.

In [16]:
trial_itt.load_expanded_data(seed = 69, p_control = 0.5)

## 8. Fit Marginal Structural Model

Use the `fit_msm()` to fit the outcome model.

In [17]:
trial_itt.fit_msm(
    weight_cols=["weight", "sample_weight"],
    modify_weights=lambda w: w.clip(upper=w.quantile(0.99)), # winsorization of extreme weights
)

Model Summary:

In [18]:
trial_itt.outcome_model

- Formula: outcome ~ 1 + assigned_treatment + 1 + followup_time + I(followup_time**2) + trial_period + I(trial_period**2) + 1 + 1 - censored  +  x2
- Treatment variable: ['assigned_treatment']
- Adjustment variables: ['censored', 'x2']
- Model fitter type: te_stats_glm_logit

Model Summary:

                 Generalized Linear Model Regression Results                  
Dep. Variable:                outcome   No. Observations:                  788
Model:                            GLM   Df Residuals:                      781
Model Family:                Binomial   Df Model:                            6
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Mon, 10 Mar 2025   Deviance:                       119.66
Time:                        00:35:44   Pearson chi2:                     456.
No. Iterations:                    29   Pseudo R-squ. (CS):                

The complete object shows all the specifications:

In [19]:
trial_itt


Trial Sequence Object
Estimand: Intent-to-Treat

Data:
 - N: 725 observations from 89 patients
     wtC        x2  age  period   wt  outcome  x1     age_s  eligible  \
0    1.0  1.146148   36       0  1.0        0   1  0.083333         1   
1    1.0  0.002200   37       1  1.0        0   1  0.166667         0   
2    1.0 -0.481762   38       2  1.0        0   0  0.250000         0   
3    1.0  0.007872   39       3  1.0        0   0  0.333333         0   
4    1.0  0.216054   40       4  1.0        0   1  0.416667         0   
..   ...       ...  ...     ...  ...      ...  ..       ...       ...   
719  1.0  1.650478   67       2  1.0        0   0  2.666667         0   
720  1.0 -0.747906   68       3  1.0        0   0  2.750000         0   
721  1.0 -0.790056   69       4  1.0        0   0  2.833333         0   
722  1.0  0.387429   70       5  1.0        0   1  2.916667         0   
723  1.0 -0.033762   71       6  1.0        0   1  3.000000         0   

           x4  censored  x3